### Semantic Kernel - Multi Agent Chat Example

In [32]:
import asyncio
import dotenv
import os
from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion import AzureChatCompletion
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.kernel import Kernel

In [33]:
dotenv.load_dotenv("..\\common\\credentials.env", verbose=True, override=True)

True

In [34]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "approved" in history[-1].content.lower()
    


In [35]:
REVIEWER_NAME = "Reviewer"
REVIEWER_INSTRUCTIONS = """
You are an reviewer who has opinions about copywriting born of a love for David Ogilvy.
The goal is to determine if the given copy is acceptable to print.
If so, state that it is approved.
If not, provide insight on how to refine suggested copy without example.
"""

COPYWRITER1_NAME = "CopyWriter1"
COPYWRITER1_INSTRUCTIONS = """
You are a copywriter with ten years of experience and are known for brevity and a dry humor.
The goal is to refine and decide on the single best copy as an expert in the field.
Only provide a single proposal per response.
You're laser focused on the goal at hand.
Don't waste time with chit chat.
Consider suggestions when refining an idea.
"""



COPYWRITER2_NAME = "CopyWriter2"
COPYWRITER2_INSTRUCTIONS = """
You are a copywriter with 0 years of experience and argues with your collegues about the solutions they provide.
The goal is to learn from collegues but also provide some solutions which you think are better.
Most of the times you are not sure about your solutions and give incorrect solutions.
Only provide a single proposal per response.
Dont consider any suggestions from team mates.
"""

In [36]:
aoai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
aoai_api_key = os.environ["AZURE_OPENAI_API_KEY"]
aoai_api_type = os.environ["OPENAI_API_TYPE"]
deployment_name = "gpt-4o" #gpt-35-turbo" # "gpt-4" 
aoai_api_version = "2024-08-01-preview" #Using latest version as of this date. Older versions may not work with the code as syntax has changed drastically.

In [37]:
def _create_kernel_with_chat_completion(service_id: str) -> Kernel:
    kernel = Kernel()
    kernel.add_service(AzureChatCompletion(service_id=service_id, api_key=aoai_api_key, endpoint=aoai_endpoint, api_version=aoai_api_version,deployment_name=deployment_name))
    return kernel

In [40]:
#async def main():
agent_reviewer = ChatCompletionAgent(
    service_id="reviewer",
    kernel=_create_kernel_with_chat_completion("reviewer"),
    name=REVIEWER_NAME,
    instructions=REVIEWER_INSTRUCTIONS,
)

agent_writer1 = ChatCompletionAgent(
    service_id="copywriter1",
    kernel=_create_kernel_with_chat_completion("copywriter1"),
    name=COPYWRITER1_NAME,
    instructions=COPYWRITER1_INSTRUCTIONS,
)
agent_writer2 = ChatCompletionAgent(
    service_id="copywriter2",
    kernel=_create_kernel_with_chat_completion("copywriter2"),
    name=COPYWRITER2_NAME,
    instructions=COPYWRITER2_INSTRUCTIONS,
)


chat = AgentGroupChat(
    agents=[agent_writer1, agent_writer2, agent_reviewer],
    termination_strategy=ApprovalTerminationStrategy(agents=[agent_reviewer], maximum_iterations=10),
)


In [41]:
chat.is_complete = False
input = "a slogan for a new line of electric cars."
#"slogan for Microsoft's SMC team working in presales"  

await chat.add_chat_message(ChatMessageContent(role=AuthorRole.USER, content=input))
print(f"# {AuthorRole.USER}: '{input}'")

async for content in chat.invoke():
    print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

print(f"# IS COMPLETE: {chat.is_complete}")

# AuthorRole.USER: 'a slogan for a new line of electric cars.'
# AuthorRole.ASSISTANT - CopyWriter1: '"Drive Tomorrow."'
# AuthorRole.ASSISTANT - CopyWriter2: '"Electrifying Your Journey Ahead."'
# AuthorRole.ASSISTANT - Reviewer: 'While "Drive Tomorrow." is concise and has a forward-looking appeal, it might not be distinctive enough to stand out in the emerging market of electric cars. The phrase "Electrifying Your Journey Ahead." aims to convey the innovative nature of the product, but it could be more impactful.

Consider refining the slogan to more clearly reflect the unique benefits and emotional appeal of the electric cars. Focus on highlighting aspects like sustainability, innovation, or the exhilarating experience of driving.

If you find a balance between emotional resonance and a clear statement of benefit, you'll craft a slogan that truly captures the spirit of your new product line.'
# AuthorRole.ASSISTANT - CopyWriter1: '"Charge Ahead."'
# AuthorRole.ASSISTANT - CopyWriter